# **Final task**

In [112]:
!pip install faker
!pip install pyspark

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 24.2 MB/s eta 0:00:00


In [114]:
import csv
from faker import Faker
import random
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
from pyspark.sql.functions import avg, sum, max, col, month, year

In [ ]:
spark = (
    SparkSession.builder.config("spark.app.name", "homework")
    .master("local[*]")
    .getOrCreate()
)

In [ ]:
df = spark.read.csv("web_server_logs.csv", header=True, inferSchema=True)

In [118]:
df.printSchema()

root
 |-- ip: string (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- method: string (nullable = true)
 |-- url: string (nullable = true)
 |-- response_code: integer (nullable = true)
 |-- response_size: integer (nullable = true)



In [ ]:
df = df.withColumn("date", df["timestamp"].cast("date"))  # Добавляем date

In [ ]:
# Задание 1. Сгруппируйте данные по IP и посчитайте количество запросов для каждого IP, выводим 10 самых активных IP.

df.groupBy("ip").agg({"url": "count"}).withColumnRenamed(
    "count(url)", "request_count"
).orderBy(col("request_count").desc()).show(10)

+---------------+-------------+
|             ip|request_count|
+---------------+-------------+
|  62.64.180.205|            2|
| 116.117.81.102|            2|
|132.224.113.177|            1|
|  218.88.73.108|            1|
|  108.55.217.78|            1|
|   99.3.191.107|            1|
| 177.250.90.166|            1|
|    8.17.184.13|            1|
|  108.84.33.128|            1|
|    202.92.29.3|            1|
+---------------+-------------+
only showing top 10 rows



In [ ]:
# Задание 2 Сгруппируйте данные по HTTP-методу и посчитайте количество запросов для каждого метода.
df.groupBy("method").agg({"url": "count"}).withColumnRenamed(
    "count(url)", "method_count"
).orderBy(col("method_count").desc()).show()

+------+------------+
|method|method_count|
+------+------------+
|  POST|       25086|
|DELETE|       24973|
|   GET|       24973|
|   PUT|       24968|
+------+------------+



In [ ]:
# Задание 3 Профильтруйте и посчитайте количество запросов с кодом ответа 404.

df.filter(col("response_code") == "404").groupBy("url").agg(
    {"url": "count"}
).withColumnRenamed("count(url)", "url_count").orderBy(col("url_count").desc()).show()

+------------------+---------+
|               url|url_count|
+------------------+---------+
|          category|      744|
|           explore|      728|
|              list|      698|
|            search|      690|
|        wp-content|      681|
|             posts|      679|
|               app|      677|
|              main|      676|
|        categories|      675|
|              tags|      669|
|               tag|      654|
|              blog|      639|
|   tags/wp-content|       87|
|explore/wp-content|       82|
|    posts/category|       73|
|    tag/categories|       72|
|          blog/app|       72|
|   search/category|       72|
|    search/explore|       72|
|     category/list|       71|
+------------------+---------+
only showing top 20 rows



In [ ]:
# Задание 4 4. Сгруппируйте данные по дате и просуммируйте размер ответов, сортируйте по дате.

df.groupBy("date").agg({"response_size": "sum"}).withColumnRenamed(
    "sum(response_size)", "total_response_size"
).orderBy(col("date").asc()).show()

+----------+-------------------+
|      date|total_response_size|
+----------+-------------------+
|2024-01-01|            1655719|
|2024-01-02|            1565031|
|2024-01-03|            1405882|
|2024-01-04|            1518123|
|2024-01-05|            1558277|
|2024-01-06|            1533007|
|2024-01-07|            1442851|
|2024-01-08|            1706504|
|2024-01-09|            1573764|
|2024-01-10|            1652017|
|2024-01-11|            1601706|
|2024-01-12|            1437884|
|2024-01-13|            1560505|
|2024-01-14|            1774513|
|2024-01-15|            1712209|
|2024-01-16|            1514433|
|2024-01-17|            1434715|
|2024-01-18|            1709194|
|2024-01-19|            1720691|
|2024-01-20|            1489896|
+----------+-------------------+
only showing top 20 rows



In [140]:
df.show()

+---------------+--------------------+------+--------------------+-------------+-------------+----------+
|             ip|           timestamp|method|                 url|response_code|response_size|      date|
+---------------+--------------------+------+--------------------+-------------+-------------+----------+
|   40.243.31.88|2024-11-15 11:58:...|   PUT|tag/wp-content/se...|          404|         4401|2024-11-15|
|  16.228.185.26|2024-10-08 19:50:...|DELETE|                tags|          200|         7554|2024-10-08|
|    36.74.27.30|2024-07-14 06:27:...|DELETE|       app/main/blog|          301|         6611|2024-07-14|
|  222.33.40.225|2024-07-21 11:28:...|   GET|                list|          301|         5869|2024-07-21|
|   27.60.108.31|2024-01-03 06:20:...|  POST|       tags/category|          301|         6460|2024-01-03|
|  110.93.33.148|2024-07-11 14:21:...|   PUT|        app/category|          301|         5278|2024-07-11|
|   14.114.25.45|2024-02-10 10:47:...|   GET| 